# **POLAR @ SemEval-2026 — Task9**
## Subtask 1: Polarization Detection    
### Authors: Sujay Nalimela 
### Date: <auto-update>

---

This notebook contains the *full pipeline* for all POLAR shared task subtasks:

### ✔ Subtask 1 — Polarization Detection (binary)  
### ⬜ Subtask 2 — Polarization Type Classification (multi-class)  
### ⬜ Subtask 3 — Polarization Manifestation Identification


In [1]:
import pandas as pd
import numpy as np
import torch
from datasets import Dataset
from sklearn.model_selection import train_test_split

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
)

from transformers.training_args import TrainingArguments

from shared.preprocess import clean_text
from shared.metrics import macro_f1

torch.backends.mps.allow_tf32 = True

print("Torch version:", torch.__version__)


/Users/sujayvarma/NLP_Project/polar-semeval-2026-task9/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Torch version: 2.9.1


We begin with **Subtask 1 (binary)** using **XLM-RoBERTa-Base**.

In [2]:
BASE_DATA = "data/subtask1"
LANG = "eng"  # change to arb, spa, amh, etc.

MODEL_NAME = "bert-base-multilingual-cased"
MAX_LEN = 128
EPOCHS = 2
BATCH_SIZE = 4

## **Load Dataset**

Each language includes:

- `train/<lang>.csv`  
- `dev/<lang>.csv`  
- `test/<lang>.csv`  

We read all splits and inspect the schema.

In [3]:
train_path = f"{BASE_DATA}/train/{LANG}.csv"
dev_path   = f"{BASE_DATA}/dev/{LANG}.csv"

train_df = pd.read_csv(train_path)
dev_df   = pd.read_csv(dev_path)

print("Train size:", len(train_df))
print("Dev size:", len(dev_df))

train_df.head()


Train size: 2676
Dev size: 133


,id,text,polarization
0,eng_973938b90b0ff5d87d35a582f83f5c89,is defending imperialism in the dnd chat,0
1,eng_07dfd4600426caca6e2c5883fcbea9ea,Still playing with this. I am now following Ra...,0
2,eng_f14519ff2302b6cd47712073f13bc461,.senate.gov Theres 3 groups out there Republic...,0
3,eng_e48b7e7542faafa544ac57b64bc80daf,"""ABC MD, David Anderson, said the additional f...",0
4,eng_7c581fb77bce8033aeba3d6dbd6273eb,"""bad people"" I have some conservative values s...",0


## **Preprocessing**

We apply light normalization (lowercasing, URL removal, emoji normalization)
using our shared `clean_text()` function.


In [4]:
train_df["text"] = train_df["text"].apply(clean_text)
dev_df["text"]   = dev_df["text"].apply(clean_text)


In [5]:
train_df_split, val_df_split = train_test_split(
    train_df,
    test_size=0.1,
    stratify=train_df["polarization"],
    random_state=42
)
train_dataset = Dataset.from_pandas(train_df)
val_dataset   = Dataset.from_pandas(val_df_split)


## **Tokenization**

We use a single tokenizer across all languages (multilingual model).


In [6]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=MAX_LEN
    )

# tokenization
train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset   = val_dataset.map(tokenize, batched=True)

# rename correct label column
train_dataset = train_dataset.rename_column("polarization", "labels")
val_dataset   = val_dataset.rename_column("polarization", "labels")

# set PyTorch formatting
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


Map: 100%|██████████| 268/268 [00:00<00:00, 21809.73 examples/s]


In [7]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
training_args = TrainingArguments(
    output_dir=f"subtask1/{LANG}_checkpoints",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=4,   # reduce load
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    fp16=False,     # MPS doesn't support FP16 fully
    bf16=False,
    logging_steps=50,
    report_to="none"
)


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=1)
    return {"macro_f1": macro_f1(preds, labels)}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

  0%|          | 0/1338 [00:00<?, ?it/s]/Users/sujayvarma/NLP_Project/polar-semeval-2026-task9/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
  4%|▎         | 50/1338 [00:34<12:31,  1.71it/s]

{'loss': 0.6748, 'grad_norm': 11.767070770263672, 'learning_rate': 2.8878923766816144e-05, 'epoch': 0.07}


  7%|▋         | 100/1338 [01:04<12:17,  1.68it/s]

{'loss': 0.6206, 'grad_norm': 8.858292579650879, 'learning_rate': 2.7757847533632287e-05, 'epoch': 0.15}


 11%|█         | 150/1338 [01:44<13:14,  1.49it/s]

...

                                                   
100%|██████████| 1338/1338 [15:34<00:00,  1.60it/s]

{'eval_loss': 0.417903333902359, 'eval_macro_f1': 0.8527363184079602, 'eval_runtime': 6.2116, 'eval_samples_per_second': 43.145, 'eval_steps_per_second': 10.786, 'epoch': 2.0}


100%|██████████| 1338/1338 [15:40<00:00,  1.42it/s]


{'train_runtime': 939.9256, 'train_samples_per_second': 5.694, 'train_steps_per_second': 1.424, 'train_loss': 0.5633094913043605, 'epoch': 2.0}


TrainOutput(global_step=1338, training_loss=0.5633094913043605, metrics={'train_runtime': 939.9256, 'train_samples_per_second': 5.694, 'train_steps_per_second': 1.424, 'total_flos': 352042592071680.0, 'train_loss': 0.5633094913043605, 'epoch': 2.0})

In [9]:
full_train_dataset = Dataset.from_pandas(train_df)

full_train_dataset = full_train_dataset.map(tokenize, batched=True)
full_train_dataset = full_train_dataset.rename_column("polarization", "labels")
full_train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

full_training_args = TrainingArguments(
    output_dir=f"subtask1/{LANG}_checkpoints",
    eval_strategy="no",       # ⛔ Disable evaluation
    save_strategy="no",       # Optional: don’t save intermediate checkpoints
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    num_train_epochs=2,
    logging_steps=50,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=full_training_args,
    train_dataset=full_train_dataset,
    tokenizer=tokenizer,
)

trainer.train()


  0%|          | 0/1338 [00:00<?, ?it/s]/Users/sujayvarma/NLP_Project/polar-semeval-2026-task9/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
  4%|▎         | 50/1338 [00:35<18:37,  1.15it/s]

{'loss': 0.4968, 'grad_norm': 0.4871438145637512, 'learning_rate': 2.8878923766816144e-05, 'epoch': 0.07}


  7%|▋         | 100/1338 [01:06<12:24,  1.66it/s]

{'loss': 0.5278, 'grad_norm': 13.145429611206055, 'learning_rate': 2.7757847533632287e-05, 'epoch': 0.15}


 11%|█         | 150/1338 [01:37<11:59,  1.65it/s]

...

 97%|█████████▋| 1300/1338 [15:10<00:25,  1.50it/s]

{'loss': 0.3612, 'grad_norm': 72.72859954833984, 'learning_rate': 8.520179372197309e-07, 'epoch': 1.94}


100%|██████████| 1338/1338 [15:37<00:00,  1.43it/s]

{'train_runtime': 937.0419, 'train_samples_per_second': 5.712, 'train_steps_per_second': 1.428, 'train_loss': 0.37606397302339606, 'epoch': 2.0}


TrainOutput(global_step=1338, training_loss=0.37606397302339606, metrics={'train_runtime': 937.0419, 'train_samples_per_second': 5.712, 'train_steps_per_second': 1.428, 'total_flos': 352042592071680.0, 'train_loss': 0.37606397302339606, 'epoch': 2.0})

In [11]:
# -------------------- Load official test data (dev/) --------------------

dev_pred_path = f"{BASE_DATA}/dev/{LANG}.csv"
test_df = pd.read_csv(dev_pred_path)

print("Loaded test (dev) size:", len(test_df))
test_df.head()

Loaded test (dev) size: 133


,id,text,polarization
0,eng_f66ca14d60851371f9720aaf4ccd9b58,God is with Ukraine and Zelensky,NaN
1,eng_3a489aa7fed9726aa8d3d4fe74c57efb,"4 Dems, 2 Republicans Luzerne County Council s...",NaN
2,eng_95770ff547ea5e48b0be00f385986483,Abuse Survivor Recounts Her Struggles at YWCA ...,NaN
3,eng_2048ae6f9aa261c48e6d777bcc5b38bf,"After Rwanda, another deportation camp disaster",NaN
4,eng_07781aa88e61e7c0a996abd1e5ea3a20,Another plea in Trump election interference probe,NaN


In [12]:
# Clean text using your preprocessing function
test_df["text"] = test_df["text"].apply(clean_text)


In [13]:
# Prepare inputs
inputs = tokenizer(
    test_df["text"].tolist(),
    truncation=True,
    padding=True,
    max_length=MAX_LEN,
    return_tensors="pt"
)

# Move to MPS or CPU
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)
inputs = {k: v.to(device) for k, v in inputs.items()}


In [14]:
# Disable gradient calculation for faster inference
with torch.no_grad():
    logits = model(**inputs).logits

# Predicted labels: 0 or 1
preds = logits.argmax(dim=1).cpu().numpy()


In [16]:
submission_df = pd.DataFrame({
    "id": test_df["id"],
    "polarization": preds.astype(int)
})

submission_df.head()


,id,polarization
0,eng_f66ca14d60851371f9720aaf4ccd9b58,0
1,eng_3a489aa7fed9726aa8d3d4fe74c57efb,0
2,eng_95770ff547ea5e48b0be00f385986483,0
3,eng_2048ae6f9aa261c48e6d777bcc5b38bf,0
4,eng_07781aa88e61e7c0a996abd1e5ea3a20,1


In [17]:
submission_df = pd.DataFrame({
    "id": test_df["id"],
    "polarization": preds.astype(int)
})

submission_df.head()


,id,polarization
0,eng_f66ca14d60851371f9720aaf4ccd9b58,0
1,eng_3a489aa7fed9726aa8d3d4fe74c57efb,0
2,eng_95770ff547ea5e48b0be00f385986483,0
3,eng_2048ae6f9aa261c48e6d777bcc5b38bf,0
4,eng_07781aa88e61e7c0a996abd1e5ea3a20,1


In [18]:
import os

SAVE_DIR = "submissions/subtask1"
os.makedirs(SAVE_DIR, exist_ok=True)

OUT_PATH = f"{SAVE_DIR}/pred_{LANG}.csv"
submission_df.to_csv(OUT_PATH, index=False)

print("Saved prediction file:", OUT_PATH)


Saved prediction file: submissions/subtask1/pred_eng.csv


In [ ]:
import shutil

zip_name = "subtask_1"   # final file will be submission_subtask1.zip
shutil.make_archive(zip_name, "zip", SAVE_DIR)

print("Created ZIP:", zip_name + ".zip")


Created ZIP: submission_subtask1.zip
